In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CPU")


Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: CPU


INFO:rdkit:Enabling RDKit 2020.09.4 jupyter extensions


In [2]:
import pickle
import os

## Check energies of repartitioned vs vanilla apo hybrid system in N501Y (14/36)

In [6]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo_0.pickle", "rb") as f:
    rhtf = pickle.load(f)

In [5]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo.pickle", "rb") as f:
    vanilla_htf = pickle.load(f)

In [10]:
# Create vanilla system
other_system = vanilla_htf.hybrid_system
positions = vanilla_htf.hybrid_positions

# Create REST system
rhtf_system = rhtf.hybrid_system


In [15]:
platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=other_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('CustomBondForce', 2.159172439102473), ('HarmonicBondForce', 298.96297195421135), ('CustomAngleForce', 6.491668007762357), ('HarmonicAngleForce', 860.6612276294761), ('CustomTorsionForce', 1307.4605962391593), ('PeriodicTorsionForce', 2440.411019741218), ('NonbondedForce', -198713.639719669), ('CustomNonbondedForce', -48.985818381082545), ('CustomBondForce', -123.33963104348693

In [16]:
# Create thermodynamic state for rest_system
thermostate_rest = ThermodynamicState(system=rhtf_system, temperature=temperature)

# Create context forrest system
integrator_rest = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_rest = thermostate_rest.create_context(integrator_rest)
context_rest.setPositions(positions)

# Get energy components for rest system
components_rest = [component[1] for component in compute_potential_components(context_rest, beta=beta)]
print(compute_potential_components(context_rest, beta=beta))

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('HarmonicBondForce', 301.1221443933138), ('HarmonicAngleForce', 867.1528956372388), ('PeriodicTorsionForce', 3747.871615980389), ('NonbondedForce', -198888.8374797606), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [17]:
assert np.isclose([components_other[0] + components_other[1]], [components_rest[0]])

In [18]:
assert np.isclose([components_other[2] + components_other[3]], [components_rest[1]])

In [19]:
assert np.isclose([components_other[4] + components_other[5]], [components_rest[2]])

In [21]:
assert np.isclose([components_other[6] + components_other[7] + components_other[8]], [components_rest[3]]), f"The energies do not match for the {components_other[6] + components_other[7] + components_other[8]} (other system) vs. {components_rest[3]} (REST system)"



AssertionError: The energies do not match for the -198885.96516909357 (other system) vs. -198888.8374797606 (REST system)

## Check energies of rest vs rhtf apo hybrid system in N501Y (14/36)

In [31]:
import mdtraj as md

In [23]:
# Read in REST snapshot
with open("/data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo_asn_1ns_snapshots.npy", "rb") as f:
    apo_cache_0 = np.load(f)

In [24]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo_0.pickle", "rb") as f:
    rhtf = pickle.load(f)

In [26]:
# Create vanilla system
other_system = rhtf.hybrid_system
positions = apo_cache_0[0]


In [33]:
# Build REST factory
for res in rhtf.hybrid_topology.residues:
    if res.resSeq == 501 and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
traj = md.Trajectory(np.array(rhtf.hybrid_positions), rhtf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, 0.2, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(rhtf.hybrid_system, solute_region=rest_atoms)

# Get REST system
REST_system = factory.REST_system

<ipython-input-33-c347861d0adf>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rest_atoms = list(md.compute_neighbors(traj, 0.2, query_indices, haystack_indices=solute_atoms)[0])
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=8.69284356, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-2.897614286199507, y=8.195691588087366, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-2.897614286199507, y=-4.097845298077956, z=7.09767740321201), unit=nanometer)]
INFO:REST:No unknown f

In [34]:
platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=other_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('HarmonicBondForce', 1062.550918063284), ('HarmonicAngleForce', 2670.6447015517324), ('PeriodicTorsionForce', 4221.775740818685), ('NonbondedForce', -250232.4123408588), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [35]:
# Create thermodynamic state for rest_system
thermostate_rest = ThermodynamicState(system=REST_system, temperature=temperature)

# Create context forrest system
integrator_rest = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_rest = thermostate_rest.create_context(integrator_rest)
context_rest.setPositions(positions)

# Get energy components for rest system
components_rest = [component[1] for component in compute_potential_components(context_rest, beta=beta)]
print(compute_potential_components(context_rest, beta=beta))

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('CustomBondForce', 1062.550918063284), ('CustomAngleForce', 2670.6447015517324), ('CustomTorsionForce', 4221.775740818685), ('NonbondedForce', -249186.5912031359), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [39]:
assert np.isclose([components_other[0]], [components_rest[0]])

In [40]:
assert np.isclose([components_other[1]], [components_rest[1]])

In [41]:
assert np.isclose([components_other[2]], [components_rest[2]])

In [46]:
assert np.isclose([components_other[3]], [components_rest[3]]), f"The energies do not match for the {components_other[3]} (other system) vs. {components_rest[3]} (REST system)"



AssertionError: The energies do not match for the -250232.4123408588 (other system) vs. -249186.5912031359 (REST system)

In [47]:
-250232.4123408588 - -249186.5912031359

-1045.8211377229018

## Check energies of rest vs vanilla apo hybrid system in N501Y (14/36)

In [48]:
import mdtraj as md

In [49]:
# Read in REST snapshot
with open("/data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo_asn_1ns_snapshots.npy", "rb") as f:
    apo_cache_0 = np.load(f)

In [50]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo.pickle", "rb") as f:
    vanilla_htf = pickle.load(f)

In [51]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo_0.pickle", "rb") as f:
    rhtf = pickle.load(f)

In [52]:
# Create vanilla system
other_system = vanilla_htf.hybrid_system
positions = apo_cache_0[0]


In [53]:
# Build REST factory
for res in rhtf.hybrid_topology.residues:
    if res.resSeq == 501 and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
traj = md.Trajectory(np.array(rhtf.hybrid_positions), rhtf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, 0.2, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(rhtf.hybrid_system, solute_region=rest_atoms)

# Get REST system
REST_system = factory.REST_system

<ipython-input-53-c347861d0adf>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rest_atoms = list(md.compute_neighbors(traj, 0.2, query_indices, haystack_indices=solute_atoms)[0])
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=8.69284356, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-2.897614286199507, y=8.195691588087366, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-2.897614286199507, y=-4.097845298077956, z=7.09767740321201), unit=nanometer)]
INFO:REST:No unknown f

In [54]:
platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=other_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('CustomBondForce', 11.677492754927671), ('HarmonicBondForce', 1050.8734253083564), ('CustomAngleForce', 17.161454004189466), ('HarmonicAngleForce', 2653.483247547543), ('CustomTorsionForce', 1602.775270759907), ('PeriodicTorsionForce', 2619.0004700587624), ('NonbondedForce', -250057.83302721006), ('CustomNonbondedForce', -46.86441047507895), ('CustomBondForce', -124.84259248507

In [55]:
# Create thermodynamic state for rest_system
thermostate_rest = ThermodynamicState(system=REST_system, temperature=temperature)

# Create context for rest system
integrator_rest = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_rest = thermostate_rest.create_context(integrator_rest)
context_rest.setPositions(positions)

# Get energy components for rest system
components_rest = [component[1] for component in compute_potential_components(context_rest, beta=beta)]
print(compute_potential_components(context_rest, beta=beta))

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('CustomBondForce', 1062.550918063284), ('CustomAngleForce', 2670.6447015517324), ('CustomTorsionForce', 4221.775740818685), ('NonbondedForce', -249186.5912031359), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [56]:
assert np.isclose([components_other[0] + components_other[1]], [components_rest[0]])

In [57]:
assert np.isclose([components_other[2] + components_other[3]], [components_rest[1]])

In [58]:
assert np.isclose([components_other[4] + components_other[5]], [components_rest[2]])

In [59]:
assert np.isclose([components_other[6] + components_other[7] + components_other[8]], [components_rest[3]]), f"The energies do not match for the {components_other[6] + components_other[7] + components_other[8]} (other system) vs. {components_rest[3]} (REST system)"



AssertionError: The energies do not match for the -250229.5400301702 (other system) vs. -249186.5912031359 (REST system)

In [60]:
-250229.5400301702 - -249186.5912031359

-1042.948827034299

## Check energies of rest vs vanilla apo hybrid system in K417N (14/38)

In [4]:
import mdtraj as md

In [6]:
# Read in REST snapshot
with open("/data/chodera/zhangi/perses_benchmark/neq/14/38/38_apo_lys_1ns_snapshots.npy", "rb") as f:
    apo_cache_0 = np.load(f)

In [7]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/38/38_apo.pickle", "rb") as f:
    vanilla_htf = pickle.load(f)

In [8]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/38/38_apo_0.pickle", "rb") as f:
    rhtf = pickle.load(f)

In [9]:
# Create vanilla system
other_system = vanilla_htf.hybrid_system
positions = apo_cache_0[0]


In [11]:
# Build REST factory
for res in rhtf.hybrid_topology.residues:
    if res.resSeq == 417 and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
traj = md.Trajectory(np.array(rhtf.hybrid_positions), rhtf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, 0.3, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(rhtf.hybrid_system, solute_region=rest_atoms)

# Get REST system
REST_system = factory.REST_system

<ipython-input-11-f94a2a44d934>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rest_atoms = list(md.compute_neighbors(traj, 0.3, query_indices, haystack_indices=solute_atoms)[0])
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=8.69284356, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-2.897614286199507, y=8.195691588087366, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-2.897614286199507, y=-4.097845298077956, z=7.09767740321201), unit=nanometer)]
INFO:REST:No unknown f

In [12]:
platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=other_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('CustomBondForce', 6.944604368309884), ('HarmonicBondForce', 1069.0400126704355), ('CustomAngleForce', 23.914457950962348), ('HarmonicAngleForce', 2561.2803319808922), ('CustomTorsionForce', 1580.68196789007), ('PeriodicTorsionForce', 2630.576915806866), ('NonbondedForce', -250130.53537660764), ('CustomNonbondedForce', -64.86937001264604), ('CustomBondForce', 42.5485937931011),

In [13]:
# Create thermodynamic state for rest_system
thermostate_rest = ThermodynamicState(system=REST_system, temperature=temperature)

# Create context for rest system
integrator_rest = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_rest = thermostate_rest.create_context(integrator_rest)
context_rest.setPositions(positions)

# Get energy components for rest system
components_rest = [component[1] for component in compute_potential_components(context_rest, beta=beta)]
print(compute_potential_components(context_rest, beta=beta))

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('CustomBondForce', 1075.984617038745), ('CustomAngleForce', 2585.1947899318557), ('CustomTorsionForce', 4211.25888369694), ('NonbondedForce', -249110.61121275875), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [14]:
assert np.isclose([components_other[0] + components_other[1]], [components_rest[0]])

In [15]:
assert np.isclose([components_other[2] + components_other[3]], [components_rest[1]])

In [16]:
assert np.isclose([components_other[4] + components_other[5]], [components_rest[2]])

In [17]:
assert np.isclose([components_other[6] + components_other[7] + components_other[8]], [components_rest[3]]), f"The energies do not match for the {components_other[6] + components_other[7] + components_other[8]} (other system) vs. {components_rest[3]} (REST system)"



AssertionError: The energies do not match for the -250152.8561528272 (other system) vs. -249110.61121275875 (REST system)

In [18]:
-250152.85 - -249110.611212

-1042.238788000017

## Check energies of rest vs vanilla apo hybrid system in N501Y (14/36)

Experiments: 
- Turn off nonbonded for glycans -- nonbonded discrepancy is now -1027
- Turn off LJ only -- no discrepancy
- Turn off charges only -- nonbonded discrepancy is -1045
- Turn off exception LJ only -- nonbonded discrepancy is -1045
- Turn off standard LJs only -- no discrepancy
- Turn off standard LJs for core atoms only -- nonbonded discrepancy is -1043
- Turn off standard LJs for unique old/new atoms only -- nonbonded discrepancy is -1045
- Turnn off standard LJs for environment atoms only -- no discrepancy

In [3]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CPU")


In [4]:
import pickle
import os

In [5]:
import mdtraj as md

In [6]:
# Read in REST snapshot
with open("/data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo_asn_1ns_snapshots.npy", "rb") as f:
    apo_cache_0 = np.load(f)

In [7]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo.pickle", "rb") as f:
    vanilla_htf = pickle.load(f)

In [8]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo_0.pickle", "rb") as f:
    rhtf = pickle.load(f)

In [9]:
rhtf.hybrid_system.getForce(4).getUseDispersionCorrection()

True

In [10]:
vanilla_htf.hybrid_system.getForce(7).getUseDispersionCorrection()

True

In [11]:
vanilla_htf.hybrid_system.getForce(8).getUseLongRangeCorrection()

False

In [7]:
# glycans = []
# for atom in rhtf.hybrid_topology.atoms:
#     if atom.residue.name in ['UYB', '4YB', 'VMB', '2MA', '0YB', '0fA', '0LB']:
#         print(atom.index, atom)
#         glycans.append(atom.index)

In [27]:
# Create vanilla system
other_system = rhtf.hybrid_system
positions = apo_cache_0[0]


In [28]:
# Set dispersion correction = False in rhtf
rhtf.hybrid_system.getForce(4).setUseDispersionCorrection(False)

In [9]:
nb_force = other_system.getForce(4)
for i in range(nb_force.getNumParticles()):
    charge, sigma, epsilon = nb_force.getParticleParameters(i)
#     # Set glycan charge and epsilon to 0
#     if i in glycans:
#         nb_force.setParticleParameters(i, charge*0, sigma, epsilon*0)
    
#     # Set LJs to 0
#     nb_force.setParticleParameters(i, charge, sigma, epsilon*0)

#     # Set charges to 0
#     nb_force.setParticleParameters(i, charge*0, sigma, epsilon)
    
#     # Set LJs to 0 for core atoms
#     if i in rhtf._atom_classes['core_atoms']:
#         nb_force.setParticleParameters(i, charge, sigma, epsilon*0)
    
#     # Set LJs to 0 for unique old/new atoms
#     if i in list(rhtf._atom_classes['unique_old_atoms']) + list(rhtf._atom_classes['unique_new_atoms']):
#         nb_force.setParticleParameters(i, charge, sigma, epsilon*0)
    
    # Set LJs to 0 for unique old/new atoms
    if i in list(rhtf._atom_classes['environment_atoms']):
        nb_force.setParticleParameters(i, charge, sigma, epsilon*0)

In [10]:
for i in range(nb_force.getNumExceptions()):
    p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(i)
#     # Set glycan charge and epsilon to 0
#     if p1 in glycans or p2 in glycans:
#         nb_force.setExceptionParameters(i, p1, p2, chargeProd*0, sigma, epsilon*0)
    
#     # Set LJs to 0
#     nb_force.setExceptionParameters(i, p1, p2, chargeProd, sigma, epsilon*0)

#     # Set charges to 0
#     nb_force.setExceptionParameters(i, p1, p2, chargeProd*0, sigma, epsilon)

In [29]:
# Build REST factory
for res in rhtf.hybrid_topology.residues:
    if res.resSeq == 501 and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
traj = md.Trajectory(np.array(rhtf.hybrid_positions), rhtf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, 0.2, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(rhtf.hybrid_system, solute_region=rest_atoms)

# Get REST system
REST_system = factory.REST_system

<ipython-input-29-c347861d0adf>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rest_atoms = list(md.compute_neighbors(traj, 0.2, query_indices, haystack_indices=solute_atoms)[0])
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=8.69284356, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-2.897614286199507, y=8.195691588087366, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-2.897614286199507, y=-4.097845298077956, z=7.09767740321201), unit=nanometer)]
INFO:REST:No unknown f

In [30]:
rhtf.hybrid_system.getForce(4).getUseDispersionCorrection()

False

In [31]:
REST_system.getForce(4).getUseDispersionCorrection()

False

In [15]:
vanilla_htf.hybrid_system.getForce(7).getUseDispersionCorrection()

True

In [24]:
vanilla_htf.hybrid_system.getForce(8).getUseLongRangeCorrection()

False

In [32]:
platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=other_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('HarmonicBondForce', 1062.550918063284), ('HarmonicAngleForce', 2670.6447015517324), ('PeriodicTorsionForce', 4221.775740818685), ('NonbondedForce', -249186.5912031359), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [33]:
# Create thermodynamic state for rest_system
thermostate_rest = ThermodynamicState(system=REST_system, temperature=temperature)

# Create context for rest system
integrator_rest = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_rest = thermostate_rest.create_context(integrator_rest)
context_rest.setPositions(positions)

# Get energy components for rest system
components_rest = [component[1] for component in compute_potential_components(context_rest, beta=beta)]
print(compute_potential_components(context_rest, beta=beta))

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('CustomBondForce', 1062.550918063284), ('CustomAngleForce', 2670.6447015517324), ('CustomTorsionForce', 4221.775740818685), ('NonbondedForce', -249186.5912031359), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [34]:
assert np.isclose([components_other[0]], [components_rest[0]])

In [35]:
assert np.isclose([components_other[1]], [components_rest[1]])

In [36]:
assert np.isclose([components_other[2]], [components_rest[2]])

In [37]:
assert np.isclose([components_other[3]], [components_rest[3]]), f"The energies do not match for the {components_other[3]} (other system) vs. {components_rest[3]} (REST system)"



In [49]:
-251218.06773276077 - -250190.706

-1027.3617327607644

In [18]:
33892.386298708036 - 34938.207436430945

-1045.821137722909

In [18]:
-251544.0139776926 - -250498.19283996968

-1045.8211377229309

In [20]:
-250197.04 - -249153.48

-1043.5599999999977

## Check energies of rest vs vanilla apo hybrid system in N501Y (14/16) -- no counterion fix

In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CPU")


Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: CPU


INFO:rdkit:Enabling RDKit 2020.09.4 jupyter extensions


In [2]:
import pickle
import os

In [3]:
import mdtraj as md

In [4]:
# Read in REST snapshot
with open("/data/chodera/zhangi/perses_benchmark/neq/14/16/16_apo_asn_1ns_snapshots.npy", "rb") as f:
    apo_cache_0 = np.load(f)

In [5]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/16/16_apo_0.pickle", "rb") as f:
    rhtf = pickle.load(f)

In [6]:
# Create vanilla system
other_system = rhtf.hybrid_system
positions = apo_cache_0[0]


In [7]:
# Build REST factory
for res in rhtf.hybrid_topology.residues:
    if res.resSeq == 501 and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
traj = md.Trajectory(np.array(rhtf.hybrid_positions), rhtf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, 0.2, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(rhtf.hybrid_system, solute_region=rest_atoms)

# Get REST system
REST_system = factory.REST_system

<ipython-input-7-c347861d0adf>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rest_atoms = list(md.compute_neighbors(traj, 0.2, query_indices, haystack_indices=solute_atoms)[0])
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.7354839, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=9.7951153, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=9.252200700000001), unit=nanometer)]
INFO:REST:No unknown forces.


In [8]:
platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=other_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('HarmonicBondForce', 1056.3819895857994), ('HarmonicAngleForce', 2561.9774845107236), ('PeriodicTorsionForce', 4192.156662248346), ('NonbondedForce', -310883.2346611991), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [9]:
# Create thermodynamic state for rest_system
thermostate_rest = ThermodynamicState(system=REST_system, temperature=temperature)

# Create context for rest system
integrator_rest = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_rest = thermostate_rest.create_context(integrator_rest)
context_rest.setPositions(positions)

# Get energy components for rest system
components_rest = [component[1] for component in compute_potential_components(context_rest, beta=beta)]
print(compute_potential_components(context_rest, beta=beta))

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('CustomBondForce', 1056.3819895857994), ('CustomAngleForce', 2561.9774845107236), ('CustomTorsionForce', 4192.156662248346), ('NonbondedForce', -309624.0562052259), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [10]:
assert np.isclose([components_other[0]], [components_rest[0]])

In [11]:
assert np.isclose([components_other[1]], [components_rest[1]])

In [12]:
assert np.isclose([components_other[2]], [components_rest[2]])

In [13]:
assert np.isclose([components_other[3]], [components_rest[3]]), f"The energies do not match for the {components_other[3]} (other system) vs. {components_rest[3]} (REST system)"



AssertionError: The energies do not match for the -310883.2346611991 (other system) vs. -309624.0562052259 (REST system)

In [16]:
len(rhtf._atom_classes['environment_atoms'])

58598

In [18]:
rhtf._atom_classes['core_atoms']

{2591,
 2592,
 2593,
 2594,
 2595,
 2596,
 2597,
 2598,
 2599,
 2600,
 2601,
 2602,
 2603,
 2604,
 2605,
 2606,
 2607,
 2608,
 2617,
 2618,
 2619,
 2620,
 2621,
 2622,
 2623,
 2624,
 2625}

In [20]:
for atom in rhtf.hybrid_topology.atoms:
    if atom.index in rhtf._atom_classes['environment_atoms']:
        print(atom)

ACE332-H1
ACE332-CH3
ACE332-H2
ACE332-H3
ACE332-C
ACE332-O
THR333-N
THR333-H
THR333-CA
THR333-HA
THR333-CB
THR333-HB
THR333-CG2
THR333-HG21
THR333-HG22
THR333-HG23
THR333-OG1
THR333-HG1
THR333-C
THR333-O
ASN334-N
ASN334-H
ASN334-CA
ASN334-HA
ASN334-CB
ASN334-HB2
ASN334-HB3
ASN334-CG
ASN334-OD1
ASN334-ND2
ASN334-HD21
ASN334-HD22
ASN334-C
ASN334-O
LEU335-N
LEU335-H
LEU335-CA
LEU335-HA
LEU335-CB
LEU335-HB2
LEU335-HB3
LEU335-CG
LEU335-HG
LEU335-CD1
LEU335-HD11
LEU335-HD12
LEU335-HD13
LEU335-CD2
LEU335-HD21
LEU335-HD22
LEU335-HD23
LEU335-C
LEU335-O
CYS336-N
CYS336-H
CYS336-CA
CYS336-HA
CYS336-CB
CYS336-HB2
CYS336-HB3
CYS336-SG
CYS336-C
CYS336-O
PRO337-N
PRO337-CD
PRO337-HD2
PRO337-HD3
PRO337-CG
PRO337-HG2
PRO337-HG3
PRO337-CB
PRO337-HB2
PRO337-HB3
PRO337-CA
PRO337-HA
PRO337-C
PRO337-O
PHE338-N
PHE338-H
PHE338-CA
PHE338-HA
PHE338-CB
PHE338-HB2
PHE338-HB3
PHE338-CG
PHE338-CD1
PHE338-HD1
PHE338-CE1
PHE338-HE1
PHE338-CZ
PHE338-HZ
PHE338-CE2
PHE338-HE2
PHE338-CD2
PHE338-HD2
PHE338-C
PHE338-O
GLY

HOH500-O
HOH500-H1
HOH500-H2
HOH501-O
HOH501-H1
HOH501-H2
HOH502-O
HOH502-H1
HOH502-H2
HOH503-O
HOH503-H1
HOH503-H2
HOH504-O
HOH504-H1
HOH504-H2
HOH505-O
HOH505-H1
HOH505-H2
HOH506-O
HOH506-H1
HOH506-H2
HOH507-O
HOH507-H1
HOH507-H2
HOH508-O
HOH508-H1
HOH508-H2
HOH509-O
HOH509-H1
HOH509-H2
HOH510-O
HOH510-H1
HOH510-H2
HOH511-O
HOH511-H1
HOH511-H2
HOH512-O
HOH512-H1
HOH512-H2
HOH513-O
HOH513-H1
HOH513-H2
HOH514-O
HOH514-H1
HOH514-H2
HOH515-O
HOH515-H1
HOH515-H2
HOH516-O
HOH516-H1
HOH516-H2
HOH517-O
HOH517-H1
HOH517-H2
HOH518-O
HOH518-H1
HOH518-H2
HOH519-O
HOH519-H1
HOH519-H2
HOH520-O
HOH520-H1
HOH520-H2
HOH521-O
HOH521-H1
HOH521-H2
HOH522-O
HOH522-H1
HOH522-H2
HOH523-O
HOH523-H1
HOH523-H2
HOH524-O
HOH524-H1
HOH524-H2
HOH525-O
HOH525-H1
HOH525-H2
HOH526-O
HOH526-H1
HOH526-H2
HOH527-O
HOH527-H1
HOH527-H2
HOH528-O
HOH528-H1
HOH528-H2
HOH529-O
HOH529-H1
HOH529-H2
HOH530-O
HOH530-H1
HOH530-H2
HOH531-O
HOH531-H1
HOH531-H2
HOH532-O
HOH532-H1
HOH532-H2
HOH533-O
HOH533-H1
HOH533-H2
HOH534-O
HOH53

HOH2166-H1
HOH2166-H2
HOH2167-O
HOH2167-H1
HOH2167-H2
HOH2168-O
HOH2168-H1
HOH2168-H2
HOH2169-O
HOH2169-H1
HOH2169-H2
HOH2170-O
HOH2170-H1
HOH2170-H2
HOH2171-O
HOH2171-H1
HOH2171-H2
HOH2172-O
HOH2172-H1
HOH2172-H2
HOH2173-O
HOH2173-H1
HOH2173-H2
HOH2174-O
HOH2174-H1
HOH2174-H2
HOH2175-O
HOH2175-H1
HOH2175-H2
HOH2176-O
HOH2176-H1
HOH2176-H2
HOH2177-O
HOH2177-H1
HOH2177-H2
HOH2178-O
HOH2178-H1
HOH2178-H2
HOH2179-O
HOH2179-H1
HOH2179-H2
HOH2180-O
HOH2180-H1
HOH2180-H2
HOH2181-O
HOH2181-H1
HOH2181-H2
HOH2182-O
HOH2182-H1
HOH2182-H2
HOH2183-O
HOH2183-H1
HOH2183-H2
HOH2184-O
HOH2184-H1
HOH2184-H2
HOH2185-O
HOH2185-H1
HOH2185-H2
HOH2186-O
HOH2186-H1
HOH2186-H2
HOH2187-O
HOH2187-H1
HOH2187-H2
HOH2188-O
HOH2188-H1
HOH2188-H2
HOH2189-O
HOH2189-H1
HOH2189-H2
HOH2190-O
HOH2190-H1
HOH2190-H2
HOH2191-O
HOH2191-H1
HOH2191-H2
HOH2192-O
HOH2192-H1
HOH2192-H2
HOH2193-O
HOH2193-H1
HOH2193-H2
HOH2194-O
HOH2194-H1
HOH2194-H2
HOH2195-O
HOH2195-H1
HOH2195-H2
HOH2196-O
HOH2196-H1
HOH2196-H2
HOH2197-O
HOH2197-

HOH4166-H1
HOH4166-H2
HOH4167-O
HOH4167-H1
HOH4167-H2
HOH4168-O
HOH4168-H1
HOH4168-H2
HOH4169-O
HOH4169-H1
HOH4169-H2
HOH4170-O
HOH4170-H1
HOH4170-H2
HOH4171-O
HOH4171-H1
HOH4171-H2
HOH4172-O
HOH4172-H1
HOH4172-H2
HOH4173-O
HOH4173-H1
HOH4173-H2
HOH4174-O
HOH4174-H1
HOH4174-H2
HOH4175-O
HOH4175-H1
HOH4175-H2
HOH4176-O
HOH4176-H1
HOH4176-H2
HOH4177-O
HOH4177-H1
HOH4177-H2
HOH4178-O
HOH4178-H1
HOH4178-H2
HOH4179-O
HOH4179-H1
HOH4179-H2
HOH4180-O
HOH4180-H1
HOH4180-H2
HOH4181-O
HOH4181-H1
HOH4181-H2
HOH4182-O
HOH4182-H1
HOH4182-H2
HOH4183-O
HOH4183-H1
HOH4183-H2
HOH4184-O
HOH4184-H1
HOH4184-H2
HOH4185-O
HOH4185-H1
HOH4185-H2
HOH4186-O
HOH4186-H1
HOH4186-H2
HOH4187-O
HOH4187-H1
HOH4187-H2
HOH4188-O
HOH4188-H1
HOH4188-H2
HOH4189-O
HOH4189-H1
HOH4189-H2
HOH4190-O
HOH4190-H1
HOH4190-H2
HOH4191-O
HOH4191-H1
HOH4191-H2
HOH4192-O
HOH4192-H1
HOH4192-H2
HOH4193-O
HOH4193-H1
HOH4193-H2
HOH4194-O
HOH4194-H1
HOH4194-H2
HOH4195-O
HOH4195-H1
HOH4195-H2
HOH4196-O
HOH4196-H1
HOH4196-H2
HOH4197-O
HOH4197-

HOH5499-H1
HOH5499-H2
HOH5500-O
HOH5500-H1
HOH5500-H2
HOH5501-O
HOH5501-H1
HOH5501-H2
HOH5502-O
HOH5502-H1
HOH5502-H2
HOH5503-O
HOH5503-H1
HOH5503-H2
HOH5504-O
HOH5504-H1
HOH5504-H2
HOH5505-O
HOH5505-H1
HOH5505-H2
HOH5506-O
HOH5506-H1
HOH5506-H2
HOH5507-O
HOH5507-H1
HOH5507-H2
HOH5508-O
HOH5508-H1
HOH5508-H2
HOH5509-O
HOH5509-H1
HOH5509-H2
HOH5510-O
HOH5510-H1
HOH5510-H2
HOH5511-O
HOH5511-H1
HOH5511-H2
HOH5512-O
HOH5512-H1
HOH5512-H2
HOH5513-O
HOH5513-H1
HOH5513-H2
HOH5514-O
HOH5514-H1
HOH5514-H2
HOH5515-O
HOH5515-H1
HOH5515-H2
HOH5516-O
HOH5516-H1
HOH5516-H2
HOH5517-O
HOH5517-H1
HOH5517-H2
HOH5518-O
HOH5518-H1
HOH5518-H2
HOH5519-O
HOH5519-H1
HOH5519-H2
HOH5520-O
HOH5520-H1
HOH5520-H2
HOH5521-O
HOH5521-H1
HOH5521-H2
HOH5522-O
HOH5522-H1
HOH5522-H2
HOH5523-O
HOH5523-H1
HOH5523-H2
HOH5524-O
HOH5524-H1
HOH5524-H2
HOH5525-O
HOH5525-H1
HOH5525-H2
HOH5526-O
HOH5526-H1
HOH5526-H2
HOH5527-O
HOH5527-H1
HOH5527-H2
HOH5528-O
HOH5528-H1
HOH5528-H2
HOH5529-O
HOH5529-H1
HOH5529-H2
HOH5530-O
HOH5530-

...

HOH9499-O
HOH9499-H1
HOH9499-H2
HOH9500-O
HOH9500-H1
HOH9500-H2
HOH9501-O
HOH9501-H1
HOH9501-H2
HOH9502-O
HOH9502-H1
HOH9502-H2
HOH9503-O
HOH9503-H1
HOH9503-H2
HOH9504-O
HOH9504-H1
HOH9504-H2
HOH9505-O
HOH9505-H1
HOH9505-H2
HOH9506-O
HOH9506-H1
HOH9506-H2
HOH9507-O
HOH9507-H1
HOH9507-H2
HOH9508-O
HOH9508-H1
HOH9508-H2
HOH9509-O
HOH9509-H1
HOH9509-H2
HOH9510-O
HOH9510-H1
HOH9510-H2
HOH9511-O
HOH9511-H1
HOH9511-H2
HOH9512-O
HOH9512-H1
HOH9512-H2
HOH9513-O
HOH9513-H1
HOH9513-H2
HOH9514-O
HOH9514-H1
HOH9514-H2
HOH9515-O
HOH9515-H1
HOH9515-H2
HOH9516-O
HOH9516-H1
HOH9516-H2
HOH9517-O
HOH9517-H1
HOH9517-H2
HOH9518-O
HOH9518-H1
HOH9518-H2
HOH9519-O
HOH9519-H1
HOH9519-H2
HOH9520-O
HOH9520-H1
HOH9520-H2
HOH9521-O
HOH9521-H1
HOH9521-H2
HOH9522-O
HOH9522-H1
HOH9522-H2
HOH9523-O
HOH9523-H1
HOH9523-H2
HOH9524-O
HOH9524-H1
HOH9524-H2
HOH9525-O
HOH9525-H1
HOH9525-H2
HOH9526-O
HOH9526-H1
HOH9526-H2
HOH9527-O
HOH9527-H1
HOH9527-H2
HOH9528-O
HOH9528-H1
HOH9528-H2
HOH9529-O
HOH9529-H1
HOH9529-H2
HOH9530-

HOH11499-O
HOH11499-H1
HOH11499-H2
HOH11500-O
HOH11500-H1
HOH11500-H2
HOH11501-O
HOH11501-H1
HOH11501-H2
HOH11502-O
HOH11502-H1
HOH11502-H2
HOH11503-O
HOH11503-H1
HOH11503-H2
HOH11504-O
HOH11504-H1
HOH11504-H2
HOH11505-O
HOH11505-H1
HOH11505-H2
HOH11506-O
HOH11506-H1
HOH11506-H2
HOH11507-O
HOH11507-H1
HOH11507-H2
HOH11508-O
HOH11508-H1
HOH11508-H2
HOH11509-O
HOH11509-H1
HOH11509-H2
HOH11510-O
HOH11510-H1
HOH11510-H2
HOH11511-O
HOH11511-H1
HOH11511-H2
HOH11512-O
HOH11512-H1
HOH11512-H2
HOH11513-O
HOH11513-H1
HOH11513-H2
HOH11514-O
HOH11514-H1
HOH11514-H2
HOH11515-O
HOH11515-H1
HOH11515-H2
HOH11516-O
HOH11516-H1
HOH11516-H2
HOH11517-O
HOH11517-H1
HOH11517-H2
HOH11518-O
HOH11518-H1
HOH11518-H2
HOH11519-O
HOH11519-H1
HOH11519-H2
HOH11520-O
HOH11520-H1
HOH11520-H2
HOH11521-O
HOH11521-H1
HOH11521-H2
HOH11522-O
HOH11522-H1
HOH11522-H2
HOH11523-O
HOH11523-H1
HOH11523-H2
HOH11524-O
HOH11524-H1
HOH11524-H2
HOH11525-O
HOH11525-H1
HOH11525-H2
HOH11526-O
HOH11526-H1
HOH11526-H2
HOH11527-O
HOH11527-

HOH13498-H2
HOH13499-O
HOH13499-H1
HOH13499-H2
HOH13500-O
HOH13500-H1
HOH13500-H2
HOH13501-O
HOH13501-H1
HOH13501-H2
HOH13502-O
HOH13502-H1
HOH13502-H2
HOH13503-O
HOH13503-H1
HOH13503-H2
HOH13504-O
HOH13504-H1
HOH13504-H2
HOH13505-O
HOH13505-H1
HOH13505-H2
HOH13506-O
HOH13506-H1
HOH13506-H2
HOH13507-O
HOH13507-H1
HOH13507-H2
HOH13508-O
HOH13508-H1
HOH13508-H2
HOH13509-O
HOH13509-H1
HOH13509-H2
HOH13510-O
HOH13510-H1
HOH13510-H2
HOH13511-O
HOH13511-H1
HOH13511-H2
HOH13512-O
HOH13512-H1
HOH13512-H2
HOH13513-O
HOH13513-H1
HOH13513-H2
HOH13514-O
HOH13514-H1
HOH13514-H2
HOH13515-O
HOH13515-H1
HOH13515-H2
HOH13516-O
HOH13516-H1
HOH13516-H2
HOH13517-O
HOH13517-H1
HOH13517-H2
HOH13518-O
HOH13518-H1
HOH13518-H2
HOH13519-O
HOH13519-H1
HOH13519-H2
HOH13520-O
HOH13520-H1
HOH13520-H2
HOH13521-O
HOH13521-H1
HOH13521-H2
HOH13522-O
HOH13522-H1
HOH13522-H2
HOH13523-O
HOH13523-H1
HOH13523-H2
HOH13524-O
HOH13524-H1
HOH13524-H2
HOH13525-O
HOH13525-H1
HOH13525-H2
HOH13526-O
HOH13526-H1
HOH13526-H2
HOH13527

HOH15498-H2
HOH15499-O
HOH15499-H1
HOH15499-H2
HOH15500-O
HOH15500-H1
HOH15500-H2
HOH15501-O
HOH15501-H1
HOH15501-H2
HOH15502-O
HOH15502-H1
HOH15502-H2
HOH15503-O
HOH15503-H1
HOH15503-H2
HOH15504-O
HOH15504-H1
HOH15504-H2
HOH15505-O
HOH15505-H1
HOH15505-H2
HOH15506-O
HOH15506-H1
HOH15506-H2
HOH15507-O
HOH15507-H1
HOH15507-H2
HOH15508-O
HOH15508-H1
HOH15508-H2
HOH15509-O
HOH15509-H1
HOH15509-H2
HOH15510-O
HOH15510-H1
HOH15510-H2
HOH15511-O
HOH15511-H1
HOH15511-H2
HOH15512-O
HOH15512-H1
HOH15512-H2
HOH15513-O
HOH15513-H1
HOH15513-H2
HOH15514-O
HOH15514-H1
HOH15514-H2
HOH15515-O
HOH15515-H1
HOH15515-H2
HOH15516-O
HOH15516-H1
HOH15516-H2
HOH15517-O
HOH15517-H1
HOH15517-H2
HOH15518-O
HOH15518-H1
HOH15518-H2
HOH15519-O
HOH15519-H1
HOH15519-H2
HOH15520-O
HOH15520-H1
HOH15520-H2
HOH15521-O
HOH15521-H1
HOH15521-H2
HOH15522-O
HOH15522-H1
HOH15522-H2
HOH15523-O
HOH15523-H1
HOH15523-H2
HOH15524-O
HOH15524-H1
HOH15524-H2
HOH15525-O
HOH15525-H1
HOH15525-H2
HOH15526-O
HOH15526-H1
HOH15526-H2
HOH15527

HOH17665-O
HOH17665-H1
HOH17665-H2
HOH17666-O
HOH17666-H1
HOH17666-H2
HOH17667-O
HOH17667-H1
HOH17667-H2
HOH17668-O
HOH17668-H1
HOH17668-H2
HOH17669-O
HOH17669-H1
HOH17669-H2
HOH17670-O
HOH17670-H1
HOH17670-H2
HOH17671-O
HOH17671-H1
HOH17671-H2
HOH17672-O
HOH17672-H1
HOH17672-H2
HOH17673-O
HOH17673-H1
HOH17673-H2
HOH17674-O
HOH17674-H1
HOH17674-H2
HOH17675-O
HOH17675-H1
HOH17675-H2
HOH17676-O
HOH17676-H1
HOH17676-H2
HOH17677-O
HOH17677-H1
HOH17677-H2
HOH17678-O
HOH17678-H1
HOH17678-H2
HOH17679-O
HOH17679-H1
HOH17679-H2
HOH17680-O
HOH17680-H1
HOH17680-H2
HOH17681-O
HOH17681-H1
HOH17681-H2
HOH17682-O
HOH17682-H1
HOH17682-H2
HOH17683-O
HOH17683-H1
HOH17683-H2
HOH17684-O
HOH17684-H1
HOH17684-H2
HOH17685-O
HOH17685-H1
HOH17685-H2
HOH17686-O
HOH17686-H1
HOH17686-H2
HOH17687-O
HOH17687-H1
HOH17687-H2
HOH17688-O
HOH17688-H1
HOH17688-H2
HOH17689-O
HOH17689-H1
HOH17689-H2
HOH17690-O
HOH17690-H1
HOH17690-H2
HOH17691-O
HOH17691-H1
HOH17691-H2
HOH17692-O
HOH17692-H1
HOH17692-H2
HOH17693-O
HOH17693-

In [23]:
nb_force = other_system.getForce(4)
nb_force_rest = REST_system.getForce(4)
for i in range(nb_force.getNumParticles()):
    charge, sigma, epsilon = nb_force.getParticleParameters(i)
    charge_rest, sigma_rest, epsilon_rest = nb_force_rest.getParticleParameters(i)
    if i in list(rhtf._atom_classes['environment_atoms']):
        if sigma != sigma_rest:
            print(i, sigma, sigma_rest)

## Check energies of rest vs vanilla apo hybrid system in N501Y (14/64) with use_dispersion_correction = True

In [3]:
import mdtraj as md

In [6]:
# Read in REST snapshot
with open("/data/chodera/zhangi/perses_benchmark/neq/14/64/64_complex_asn_1ns_snapshots.npy", "rb") as f:
    apo_cache_0 = np.load(f)

In [9]:
with open("/data/chodera/zhangi/perses_benchmark/neq/15/64/64_complex.pickle", "rb") as f:
    vanilla_htf = pickle.load(f)

In [22]:
vanilla_htf.hybrid_system.getForce(7).getUseDispersionCorrection()

True

In [24]:
vanilla_htf.hybrid_system.getForce(8).getUseLongRangeCorrection()

False

In [8]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/64/64_complex_0.pickle", "rb") as f:
    rhtf = pickle.load(f)

In [10]:
# Create vanilla system
other_system = vanilla_htf.hybrid_system
positions = apo_cache_0[0]


In [27]:
# Build REST factory
for res in rhtf.hybrid_topology.residues:
    if res.resSeq == 501 and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
traj = md.Trajectory(np.array(rhtf.hybrid_positions), rhtf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, 0.5, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(rhtf.hybrid_system, solute_region=rest_atoms, use_dispersion_correction=True)

# Get REST system
REST_system = factory.REST_system

<ipython-input-27-a96e2261c163>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rest_atoms = list(md.compute_neighbors(traj, 0.5, query_indices, haystack_indices=solute_atoms)[0])
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=13.804333500000002, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-4.601444128722184, y=13.014850568080696, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-4.601444128722184, y=-6.507424496441187, z=11.271191673136768), unit=nanometer)]
INFO:REST:N

In [28]:
REST_system.getForce(4).getUseDispersionCorrection()

True

In [29]:
platform = configure_platform(REFERENCE_PLATFORM)

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=other_system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(positions)

# Get energy components for non-rest system
components_other = [component[1] for component in compute_potential_components(context_other, beta=beta)]
print(compute_potential_components(context_other, beta=beta))


conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('CustomBondForce', 9.54303596822219), ('HarmonicBondForce', 5307.2751610521), ('CustomAngleForce', 22.90838455406472), ('HarmonicAngleForce', 13353.627296733646), ('CustomTorsionForce', 28.173777533525932), ('PeriodicTorsionForce', 20241.85129816155), ('NonbondedForce', -1010753.6573056893), ('CustomNonbondedForce', -69.50824615155672), ('CustomBondForce', -124.81380751549032),

In [30]:
# Create thermodynamic state for rest_system
thermostate_rest = ThermodynamicState(system=REST_system, temperature=temperature)

# Create context for rest system
integrator_rest = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context_rest = thermostate_rest.create_context(integrator_rest)
context_rest.setPositions(positions)

# Get energy components for rest system
components_rest = [component[1] for component in compute_potential_components(context_rest, beta=beta)]
print(compute_potential_components(context_rest, beta=beta))

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
[('CustomBondForce', 5316.818197020322), ('CustomAngleForce', 13376.535681287709), ('CustomTorsionForce', 20270.025075695077), ('NonbondedForce', -1010950.9527000559), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]


In [31]:
assert np.isclose([components_other[0] + components_other[1]], [components_rest[0]])

In [32]:
assert np.isclose([components_other[2] + components_other[3]], [components_rest[1]])

In [33]:
assert np.isclose([components_other[4] + components_other[5]], [components_rest[2]])

In [34]:
assert np.isclose([components_other[6] + components_other[7] + components_other[8]], [components_rest[3]]), f"The energies do not match for the {components_other[6] + components_other[7] + components_other[8]} (other system) vs. {components_rest[3]} (REST system)"

